In [1]:
import cv2
# import time
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, savgol_filter

In [2]:
# Initialize MediaPipe Face Mesh
mpDraw = mp.solutions.drawing_utils #helps draw on faces
mp_face_mesh = mp.solutions.face_mesh #face mesh model
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5) #object of face mesh model

# Drawing specifications to draw the face mesh
drawSpec = mpDraw.DrawingSpec(thickness=1, circle_radius=1, color=(0, 255, 0))

# Open the video file
video_path = r"C:\Users\USER\Documents\SLIIT\Datasets\CardioFit AI Dataset\Videos\VID_20240428_134059.mp4"
cap = cv2.VideoCapture(video_path) # Open the video file
# cap = cv2.VideoCapture(0) # Open the webcam

fps = cap.get(cv2.CAP_PROP_FPS)
time_axis = []

# Initialize lists to store ROI signals.
cheek_signals = []
nose_signals = []
forehead_signals = []

In [3]:
def displayimg(img):
    cv2.imshow('image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
# Get RGB values for the ROIs.
def extract_rgb(roi, frame):
    roi_mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    cv2.fillPoly(roi_mask, [np.array(roi)], 255)
    mean_color = cv2.mean(frame, mask=roi_mask)[:3]
    return mean_color

### Create an image with numbered landmarks

In [6]:
img = cv2.imread(r"C:\Users\USER\Documents\SLIIT\Datasets\CardioFit AI Dataset\SampleFace2.png", 1)

# Convert the image to RGB
rgb_frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

results = face_mesh.process(rgb_frame)

face = []

if results.multi_face_landmarks:
        # for face_landmarks in results.multi_face_landmarks:
        #     for landmark in face_landmarks.landmark:
        #         x = int(landmark.x * img.shape[1])
        #         y = int(landmark.y * img.shape[0])
        #         cv2.circle(img, (x, y), 1, (0, 255, 0), -1)
        # mpDraw.draw_landmarks(img, results.multi_face_landmarks[0], mp_face_mesh.FACEMESH_CONTOURS,
        #                                    drawSpec, drawSpec)
                
        # identifying the landmarks
        for id, lm in enumerate(results.multi_face_landmarks[
                                    0].landmark):  #enumerate() returns key values pairs of the landmark id and the landmark itself
            height, width, channels = img.shape
            x, y = int(lm.x * width), int(lm.y * height)  #this is done to undo the normalizing and obtain the pixel values
    
            # prints the id of the landmark in the exact pixel location
            cv2.putText(img, str(id), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1)
            # print(id, x, y)
            face.append([x, y])
            
        # resizedImg = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_AREA)
        # 
        # displayimg(resizedImg)

C:\Users\USER\miniconda3\envs\mediapipe-research\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [7]:
displayimg(img)
# cv2.imwrite('numbered_landmarks.png', img)

In [4]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
    # if reading video
    if cap.get(cv2.CAP_PROP_POS_MSEC) > 10000:
        break
        
    # if reading webcam
    # if time.time() - startTime > 10:
    #     break
    
    # Convert the frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the frame to detect face landmarks
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # extract coordinates for specific landmarks
            height, width, channels = frame.shape
            landmarks = [(int(point.x * width), int(point.y * height)) for point in face_landmarks.landmark]

            # Define ROIs based on landmark indices (example indices for cheeks and nose).
            left_cheek_indices = [143, 111, 117, 118, 119, 120, 121, 128, 114, 217, 126, 142, 203, 205, 187, 123, 116]
            right_cheek_indices = [357, 350, 349, 348, 347, 346, 340, 372, 345, 352, 411, 427, 426, 423, 279, 429, 437, 343, 357]
            nose_indices = [8, 193, 244, 128, 114, 217, 209, 49, 64, 98, 218, 237, 238, 19, 274, 438, 455, 278, 279, 429, 437, 343, 412, 465, 417]
            forehead_landmarks = [9, 107, 104, 68, 54, 103, 67, 109, 10, 338, 297, 332, 284, 298, 333, 299, 336, 337]  # Example indices

            # Extract ROI signals.
            left_cheek_roi = [landmarks[i] for i in left_cheek_indices]
            right_cheek_roi = [landmarks[i] for i in right_cheek_indices]
            nose_roi = [landmarks[i] for i in nose_indices]
            forehead_roi = [landmarks[i] for i in forehead_landmarks]

            left_cheek_signal = extract_rgb(left_cheek_roi, frame)
            right_cheek_signal = extract_rgb(right_cheek_roi, frame)
            nose_signal = extract_rgb(nose_roi, frame)
            forehead_signal = extract_rgb(forehead_roi, frame)

            # Store signals.
            cheek_signals.append(np.mean([left_cheek_signal, right_cheek_signal], axis=0))
            nose_signals.append(nose_signal)
            forehead_signals.append(forehead_signal)

    # convert lists to numpy arrays
    cheek_signals = np.array(cheek_signals)
    nose_signals = np.array(nose_signals)
    forehead_signals = np.array(forehead_signals)

    # Calculate the rPPG signal (example using green channel).
    cheek_rppg_signal = cheek_signals[:, 1]  # Green channel.
    nose_rppg_signal = nose_signals[:, 1]  # Green channel.
    forehead_rppg_signal = forehead_signals[:, 1]  # Green channel.

    # Combine signals from different ROIs (simple average for this example).
    rppg_signal = np.mean([cheek_rppg_signal, nose_rppg_signal], axis=0)

    coords = [(int(face_landmarks.landmark[i].x * frame.shape[1]), int(face_landmarks.landmark[i].y * frame.shape[0])) for i in forehead_landmarks] #for each index, it retrieves the corresponding landmark from the face_landmarks object

    # Create a mask for the forehead region
    mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    cv2.fillConvexPoly(mask, np.array(coords, dtype=np.int32), 1)

    # Extract the mean color value in the forehead region
    mean_color = cv2.mean(frame, mask=mask)[:3]
    rppg_signal.append(np.mean(mean_color))  # Take the mean of the RGB values

    # Add current frame time to time axis
    time_axis.append(cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0)
            
    cap.release()
    cv2.destroyAllWindows()

C:\Users\USER\miniconda3\envs\mediapipe-research\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [11]:
# Visualize the rPPG signal.

plt.plot(rppg_signal)
plt.title('rPPG Signal')
plt.xlabel('Frame')
plt.ylabel('Amplitude')
plt.show()

NameError: name 'rppg_signal' is not defined

## Rewriting above code to apply for an image

In [18]:
frame = cv2.imread(r"C:\Users\USER\Documents\SLIIT\Datasets\CardioFit AI Dataset\SampleFace2.png", 1)

rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

# Process the frame to detect face landmarks
results = face_mesh.process(rgb_frame)
mask = None

if results.multi_face_landmarks:
    for face_landmarks in results.multi_face_landmarks:
        # Extract coordinates of landmarks corresponding to the forehead region
        # For simplicity, let's assume we use landmark indices from the forehead region
        forehead_landmarks = [10, 338, 297, 332, 284, 251, 389, 356, 454]  # Example indices
        coords = [(int(face_landmarks.landmark[i].x * frame.shape[1]), int(face_landmarks.landmark[i].y * frame.shape[0])) for i in forehead_landmarks] #for each index, it retrieves the corresponding landmark from the face_landmarks object
        
        # Create a mask for the forehead region
        mask = np.zeros(frame.shape[:2], dtype=np.uint8)
        
displayimg(mask)
cv2.fillConvexPoly(mask, np.array(coords, dtype=np.int32), 1)

# Scale up the mask for visualization
mask_visual = mask * 255

# Display the mask
cv2.imshow('Mask', mask_visual)
cv2.waitKey(0)
cv2.destroyAllWindows()
        

end of Rewriting above code to apply for an image

In [ ]:
# Convert lists to numpy arrays
rppg_signal = np.array(rppg_signal)
time_axis = np.array(time_axis)

# Smooth the rPPG signal
smoothed_signal = savgol_filter(rppg_signal, window_length=51, polyorder=3)

# Detect peaks in the smoothed rPPG signal
peaks, _ = find_peaks(smoothed_signal, distance=fps * 0.6)  # Adjust the distance parameter based on expected heart rate

# Calculate the time differences between successive peaks
peak_times = time_axis[peaks]
intervals = np.diff(peak_times)

# Calculate heart rate (in beats per minute)
heart_rate = 60.0 / intervals
average_heart_rate = np.mean(heart_rate)

# Plot the rPPG signal and detected peaks
plt.figure(figsize=(10, 4))
plt.plot(time_axis, smoothed_signal, label='Smoothed rPPG Signal')
plt.plot(peak_times, smoothed_signal[peaks], "x", label='Detected Peaks')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title('rPPG Signal with Detected Peaks')
plt.legend()
plt.grid(True)
plt.show()

# Display the calculated heart rate
print(f'Average Heart Rate: {average_heart_rate:.2f} bpm')